In [50]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
data_folder = '../datasets'

In [51]:
#file = 'ML_DATASET_Hackathon_Supervised.csv'
file = 'Version_2.xls'

In [52]:
data = pd.read_csv(os.path.join(data_folder, file))

In [53]:
cutoff = 0.98
data = data[data['percentile'] <= cutoff]

In [58]:
counts = pd.DataFrame(data.Team.value_counts()).reset_index()
non_stratifiable = list(counts[counts['Team'] < 2]['index'])
data_to_strat = data[~data.Team.isin(non_stratifiable)]
extra =  data[data.Team.isin(non_stratifiable)]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data_to_strat[['Problem_Abstract']], 
                                                    data_to_strat[['Team']], 
                                                    test_size=0.1, 
                                                    stratify=data_to_strat[['Team']])

In [10]:
x_train = pd.concat([x_train,extra[['Problem_Abstract']]], axis=0)
y_train = pd.concat([y_train,extra[['Team']]], axis=0)

In [11]:

vect = TfidfVectorizer(strip_accents='unicode',
                      stop_words='english',
                      ngram_range=(1,5))
vect.fit(x_train.fillna('').Problem_Abstract)

TfidfVectorizer(ngram_range=(1, 5), stop_words='english',
                strip_accents='unicode')

In [12]:
x_train = vect.transform(x_train.fillna('').Problem_Abstract)
x_test = vect.transform(x_test.fillna('').Problem_Abstract)

In [13]:

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

/Users/A107809368/envs/experiment/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/A107809368/envs/experiment/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
import xgboost as xgb
classifier = xgb.XGBClassifier(n_jobs=8, n_estimators=100, verbosity=3)
classifier.fit(x_train, y_train)

In [27]:
y_predicted = classifier.predict(x_test)

print(classification_report(y_predicted, y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         7
           4       0.25      1.00      0.40         1
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         0
           9       0.67      1.00      0.80         2
          10       0.50      0.33      0.40         3
          12       0.00      0.00      0.00         0
          14       0.50      0.50      0.50         2
          16       0.00      0.00      0.00         0
          19       0.00      0.00      0.00         0
          22       1.00      1.00      1.00         1
          23       0.00      0.00      0.00         0
          24       0.82      0.96      0.88        24
          25       1.00      1.00      1.00         1
          26       0.44    

/Users/A107809368/envs/experiment/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/A107809368/envs/experiment/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/A107809368/envs/experiment/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users

### using embeddings

In [59]:
import numpy as np
emb_path = '../data/embeddings_mpnet.npy'
embedding = np.load(emb_path)

In [60]:
y = data_to_strat[['Team']]

In [61]:
y_train, y_test = train_test_split(y, test_size=0.1, stratify=y['Team'])

In [62]:
x_train = embedding[y_train.index]
x_test = embedding[y_test.index]

x_train = np.concatenate((x_train,embedding[extra['Problem_Abstract'].index]))
y_train = pd.concat([y_train,extra[['Team']]], axis=0)


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

/Users/A107809368/envs/experiment/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/A107809368/envs/experiment/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
from catboost import CatBoostClassifier

In [38]:
import xgboost as xgb
#classifier = xgb.XGBClassifier(n_jobs=8, n_estimators=100, verbosity=3)
classifier = CatBoostClassifier()
classifier.fit(x_train, y_train)

Learning rate set to 0.085206
0:	learn: 4.2979337	total: 1.86s	remaining: 30m 59s
1:	learn: 4.0595270	total: 3.54s	remaining: 29m 24s
2:	learn: 3.8660271	total: 5.25s	remaining: 29m 4s
3:	learn: 3.6707900	total: 6.97s	remaining: 28m 54s
4:	learn: 3.4988996	total: 8.68s	remaining: 28m 46s
5:	learn: 3.3413271	total: 10.4s	remaining: 28m 41s
6:	learn: 3.2368188	total: 12.1s	remaining: 28m 41s
7:	learn: 3.1413334	total: 13.9s	remaining: 28m 44s


KeyboardInterrupt: 

In [37]:
y_predicted = classifier.predict(x_test)
from sklearn.metrics import classification_report
print(classification_report(y_predicted, y_test))

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.